In [4]:
import cv2
import os
import glob
import numpy as np

In [13]:
folder_path = '/root/flatten/outputs'
video_paths = glob.glob(os.path.join(folder_path, "*"))

In [12]:
out_path = '/root/flatten/frames'

In [14]:
import cv2

def split_video(video_path, output_dir):
    # 비디오 로드
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print("비디오 파일을 열 수 없습니다.")
        return
    
    video_name = os.path.basename(video_path)[:-4]
    print(video_name)

    if video_name.split('_')[0] == "spynet":
        output_dir = os.path.join(output_dir, "spynet", video_name)
    else:
        output_dir = os.path.join(output_dir, "raft", video_name)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"폴더 '{output_dir}' 생성 완료")
    else:
        print(f"폴더 '{output_dir}' 이미 존재합니다.")

    # 프레임 수 얻기
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    print("총 프레임 수:", total_frames)
    
    # 프레임별로 반복하여 영상 자르기
    for frame_num in range(total_frames):
        # 프레임 읽기
        ret, frame = video_capture.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break
        
        # 프레임 저장
        frame_name = f"{video_name}_{frame_num}.png"
        output_path = os.path.join(output_dir, frame_name)
        cv2.imwrite(output_path, frame)
        
        # 작업 진행 상황 표시
        print(f"프레임 {frame_num}/{total_frames} 저장 완료")
    
    # 비디오 캡처 객체 해제
    video_capture.release()
    print("영상 자르기 완료")

# 입력 비디오 파일 경로
video_path = "/root/flatten/outputs/raft_A car drifts on a snowy road_None_25_25_2.mp4"
# 프레임 저장할 디렉토리 경로
output_dir = "/root/flatten/frames"

for video_path in video_paths:

    # 비디오 자르기 함수 호출
    split_video(video_path, output_dir)


spynet_Pointillism painting, detailed_None_25_25_2
폴더 '/root/flatten/frames/spynet/spynet_Pointillism painting, detailed_None_25_25_2' 생성 완료
총 프레임 수: 25
프레임 0/25 저장 완료
프레임 1/25 저장 완료
프레임 2/25 저장 완료
프레임 3/25 저장 완료
프레임 4/25 저장 완료
프레임 5/25 저장 완료
프레임 6/25 저장 완료
프레임 7/25 저장 완료
프레임 8/25 저장 완료
프레임 9/25 저장 완료
프레임 10/25 저장 완료
프레임 11/25 저장 완료
프레임 12/25 저장 완료
프레임 13/25 저장 완료
프레임 14/25 저장 완료
프레임 15/25 저장 완료
프레임 16/25 저장 완료
프레임 17/25 저장 완료
프레임 18/25 저장 완료
프레임 19/25 저장 완료
프레임 20/25 저장 완료
프레임 21/25 저장 완료
프레임 22/25 저장 완료
프레임 23/25 저장 완료
프레임 24/25 저장 완료
영상 자르기 완료
raft_A metal sculpture_None_25_25_2
폴더 '/root/flatten/frames/raft/raft_A metal sculpture_None_25_25_2' 생성 완료
총 프레임 수: 5
프레임 0/5 저장 완료
프레임 1/5 저장 완료
프레임 2/5 저장 완료
프레임 3/5 저장 완료
프레임 4/5 저장 완료
영상 자르기 완료
spynet_A metal sculpture_None_25_25_2
폴더 '/root/flatten/frames/spynet/spynet_A metal sculpture_None_25_25_2' 생성 완료
총 프레임 수: 5
프레임 0/5 저장 완료
프레임 1/5 저장 완료
프레임 2/5 저장 완료
프레임 3/5 저장 완료
프레임 4/5 저장 완료
영상 자르기 완료
raft_A detailed woolen toy cat_None_25_25

In [1]:
import cv2
import os
import glob
import numpy as np
import torch
from torchmetrics.multimodal.clip_score import CLIPScore

metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

/root/miniconda/envs/flatten/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# CLIP-T

In [19]:
# 입력 비디오 파일 경로
video_path = "/root/flatten/outputs/raft_A car drifts on a snowy road_None_25_25_2.mp4"
# 프레임 저장할 디렉토리 경로
output_dir = "/root/flatten/frames"
CLIPT_score_dict = {}

for optflow in ["raft", "spynet"]:
    CLIPT_score_dict[optflow] = 0
    frame_folder = os.path.join(output_dir, optflow)
    opt_frame_folders = glob.glob(frame_folder+"/*")
    opt_scores = []

    for opt_frame_folder in opt_frame_folders:
        
        opt_frame_files = glob.glob(os.path.join(opt_frame_folder, "*"))
        file_name = os.path.basename(opt_frame_files[0])[:-4]

        TP = file_name.split('_')[1]
        frame_n = len(opt_frame_files)
        scores = []
        for opt_frame_file in opt_frame_files:
            image = cv2.imread(str(opt_frame_file))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = np.transpose(image, (2, 0, 1))
            image = torch.tensor(image)
            score = metric(image, TP).detach()
            scores.append(score)
        
        opt_scores.append(sum(scores)/frame_n)
        print(optflow, TP, opt_scores)
    
    CLIPT_score_dict[optflow] = sum(opt_scores) / len(opt_scores)

raft A car drifts on a snowy road [tensor(32.3572)]
raft Pointillism painting, detailed [tensor(32.3572), tensor(26.5821)]
raft A cat, Van Gogh style [tensor(32.3572), tensor(26.5821), tensor(34.2844)]
raft A detailed woolen toy cat [tensor(32.3572), tensor(26.5821), tensor(34.2844), tensor(37.1612)]
raft Wooden trucks drive on a racetrack [tensor(32.3572), tensor(26.5821), tensor(34.2844), tensor(37.1612), tensor(39.3742)]
raft Several goldfish swim in a pond [tensor(32.3572), tensor(26.5821), tensor(34.2844), tensor(37.1612), tensor(39.3742), tensor(31.3948)]
raft Cartoon Style [tensor(32.3572), tensor(26.5821), tensor(34.2844), tensor(37.1612), tensor(39.3742), tensor(31.3948), tensor(23.8056)]
raft A metal sculpture [tensor(32.3572), tensor(26.5821), tensor(34.2844), tensor(37.1612), tensor(39.3742), tensor(31.3948), tensor(23.8056), tensor(30.4150)]
spynet Several goldfish swim in a pond [tensor(31.1729)]
spynet A detailed woolen toy cat [tensor(31.1729), tensor(36.7887)]
spynet W

In [4]:
# 입력 비디오 파일 경로
video_path = "/root/flatten/outputs/raft_A car drifts on a snowy road_None_25_25_2.mp4"
# 프레임 저장할 디렉토리 경로
output_dir = "/root/flatten/frames"
CLIPT_score_dict = {}

for optflow in ["raft", "spynet"]:
    CLIPT_score_dict[optflow] = 0
    frame_folder = os.path.join(output_dir, optflow)
    opt_frame_folders = glob.glob(frame_folder+"/*")
    opt_scores = []

    for opt_frame_folder in opt_frame_folders:
        
        opt_frame_files = glob.glob(os.path.join(opt_frame_folder, "*"))
        file_name = os.path.basename(opt_frame_files[0])[:-4]

        TP = file_name.split('_')[1]
        frame_n = len(opt_frame_files)
        scores = []
        for opt_frame_file in opt_frame_files:
            image = cv2.imread(str(opt_frame_file))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = np.transpose(image, (2, 0, 1))
            image = torch.tensor(image)
            score = metric(image, TP).detach()
            scores.append(score)
        
        opt_scores.append(sum(scores)/frame_n)
        print(optflow, TP, opt_scores)
    
    CLIPT_score_dict[optflow] = sum(opt_scores) / len(opt_scores)

NameError: name 'os' is not defined